In [1]:
import psycopg2

from itertools import chain
import numpy as np
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import requests 
from datetime import datetime, timedelta
import re
import pytz

In [3]:
cur=conn.cursor()

In [4]:
cur.execute("""SELECT DISTINCT(name) FROM public.cards WHERE rarity IN('mythic','rare') AND setcode IN(SELECT code FROM public.sets WHERE type IN ('expansion','core') and releasedate>=(SELECT releasedate FROM public.sets WHERE mcmname='Return to Ravnica'))""")
cardnames=cur.fetchall()


In [5]:
utc_now = pytz.utc.localize(datetime.utcnow())
pst_now = utc_now.astimezone(pytz.timezone("America/Los_Angeles"))
yesterday=pst_now - timedelta(1)

print(yesterday)
print(type(yesterday))

for cardname in cardnames:
    cur.execute('''SELECT name FROM public.cards WHERE rarity IN('mythic','rare') AND name ILIKE %s AND setcode IN(SELECT code FROM public.sets WHERE type IN ('expansion','core') and releasedate>=(SELECT releasedate FROM public.sets WHERE mcmname='Return to Ravnica'))''',('%'+cardname[0]+'%',))
    simnames=cur.fetchall()
    simnames=set(simnames)
    if len(simnames)==1:
        pass
    else:
        
        print(simnames)
        cur.execute('''SELECT id,title FROM public.transactions WHERE cardname=%s AND carddate=%s''',(str(cardname[0]),(yesterday).strftime('%Y-%m-%d')))
        transactions=cur.fetchall()
        print(cardname)
        print(len(transactions))
        if len(transactions)==0:
            pass
        else:
            
            for transid,title in transactions:
                
                for adjname in simnames:
                    if adjname[0].lower()==cardname[0].lower():
                        pass
                    elif title.lower().find(adjname[0].lower())!=-1:
                        cur.execute("""UPDATE transactions SET possiblybad=%s WHERE id=%s""",(True,transid))
                        break
            
    conn.commit()   

2020-06-28 09:19:38.438669-07:00
<class 'datetime.datetime'>
{('Assemble',), ('Assemble the Legion',), ('Assembled Alphas',)}
('Assemble',)
1
{('Beck',), ('Admiral Beckett Brass',)}
('Beck',)
0
{('Believe',), ('Silence the Believers',)}
('Believe',)
0
{('Defiant Bloodlord',), ('Bloodstained Mire',), ('Bloodsoaked Champion',), ('Blood Sun',), ("Arguel's Blood Fast",), ('Abolisher of Bloodlines',), ('Blood Crypt',), ('Blood-Chin Fanatic',), ('Exava, Rakdos Blood Witch',), ('Rageblood Shaman',), ('Vilis, Broker of Blood',), ('Sorin, Vengeful Bloodlord',), ('Bloodcrazed Paladin',), ('Blood Baron of Vizkopa',), ('Blood Scrivener',), ('Sorin, Imperious Bloodlord',), ('Bloodhall Priest',), ('Blood',), ('Purphoros, Bronze-Blooded',), ('Kazarov, Sengir Pureblood',), ('Blood Operative',), ('Sarkhan, Fireblood',), ('Razaketh, the Foulblooded',), ('Priest of the Blood Rite',)}
('Blood',)
33
{('Wildcall',), ('Dreamcaller Siren',), ('Ajani, Caller of the Pride',), ('Chord of Calling',), ('Call',), (

In [6]:
cur.close()
conn.close()